<a href="https://colab.research.google.com/github/JohnsonYu0924/114_2_text-analysis/blob/main/L5_pre_processing_%26_dateutil.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 1. 建立文字、前處理、tokenize

這裡是用三個引號 """ ... """ 放一大段原始文字，Python 看到三引號就會當成一個長字串，所以很適合貼文件、HTML、立法院紀錄這種長文本。



In [ ]:
## Example text, the header from a case file
## Triple quotes are the easiest
myText = """
                      PUBLIC BILLS AND RESOLUTIONS

  Under clause 2 of rule XII, public bills and resolutions of the
following titles were introduced and severally referred, as follows:

           By Mr. DUNN (for himself and Mr. Cunningham):
       H.R. 6092. A bill to direct the Secretary of Veterans
     Affairs to establish a national clinical pathway for prostate
     cancer, access to life-saving extending precision clinical
     trials and research, and for other purposes; to the Committee
     on Veterans' Affairs.

           By Mr. DUNN:
       H.R. 6093. A bill to amend title 38, United States Code, to
     extend the authority of the Secretary of Veterans Affairs to
     continue to pay educational assistance or subsistence
     allowances to eligible persons when educational institutions
     are closed; to the Committee on Veterans' Affairs.
           By Mr. ENGEL (for himself, Mr. McCaul, Ms. Bass, Mr.
             Smith of New Jersey, Mr. McGovern, Mr. Kildee, and
             Ms. Lee of California):
       H.R. 6094. A bill to support a civilian-led democratic
     transition, promote accountability for human rights abuses,
     and encourage fiscal transparency in Sudan, and for other
     purposes; to the Committee on Foreign Affairs, and in
     addition to the Committees on the Judiciary, and Financial
     Services, for a period to be subsequently determined by the
     Speaker, in each case for consideration of such provisions as
     fall within the jurisdiction of the committee concerned.
"""

- 匯入 re 做正規表示法處理，匯入 word_tokenize 要把文字切成一個一個 token。

- 把原文裡的 H.R. 6092 這種格式，先變成沒有點的 HR 6092。因為等一下要用「看到 HR → 抓下一個字」，如果不先統一格式，會抓不到。H\.R\. 這裡用 \. 是因為在 regex 裡面 . 是特殊符號，要跳脫。

- 把整段文字切成一個一個 token，變成 list。之後的 for loop 都是跑這個 list。

- 測試一下有沒有切成功、長得怎樣。[30:45] 只是偷看一小段。

In [ ]:
import re
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt_tab')


myText = re.sub(r"H\.R\. ", "HR ", myText)
myText2 = word_tokenize(myText)
print(myText2[30:45])
print(myText2[250])

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


['DUNN', '(', 'for', 'himself', 'and', 'Mr.', 'Cunningham', ')', ':', 'HR', '6092', '.', 'A', 'bill', 'to']
.


### 2. 抽法案號碼的基本邏輯（第一版）


- 先做一個空 list bills：等一下抓到的號碼要丟進來。

- hr_on = False 這個是「開關」(flag)：現在有沒有剛看到 HR？

- 一個一個 token 跑。上面那行是測試用的，想看「現在跑到哪個字、開關是不是開著」，所以有時候會 print。

- 如果「開關目前是關的」而且「現在這個字剛好是 HR」，就把開關打開。意思就是：下一個字就是我們要的號碼了。


- 如果「開關是開的」，代表「上一個字是 HR」，那現在這個字就是號碼，把它丟進 bills。然後印出來。最後把開關關回去，等下一個 HR。這三行的順序要照這樣，抓 → 印 → 關掉。

- ```else: pass``` 其他情況都不做事。這就是最基本的「看到 HR → 抓下一個字」寫法。

In [ ]:
## Extract the bill numbers discussed
bills = []  ## create a cart first, and then put the result of the for loop in it.
hr_on = False
for word in myText2:
    # print(word+";" + str (hr_on))
    if hr_on == False and word == "HR":  ## with boolean logic, : so if there is hr, set the hr_on to true
        hr_on = True
    elif hr_on == True:
        bills.append(word)
        print(word)   ## if you do not want it, just put # in front of it.
        hr_on = False ## this line of code means that; after we catch up word and we will turn in off.
                      ## and you have to rememver to turn in off. 是有順序的動作：抓了字，然後印了字，我們就把它關掉了。
                      ## 另外就是因為，我們已經 tokenize 這些文章了，所以一個字會是一行。每一個字都會是一個 iteration.
                      ## 然後就會是到 else: pass 那個 block。
    else:
        pass


6092
6093
6094


In [ ]:
# You don't have to use else (or elif) clauses
bills = []
hr_on = False
for word in myText2:
    if hr_on == False and word == "HR":
        hr_on = True
    elif hr_on == True:
        bills.append(word)
        print(word)
        hr_on = False

6092
6093
6094


### 3. 同一個邏輯的不同寫法（三種）

示範「if / elif / else」要怎麼拆。

寫法 1（跟上面一樣）

寫法 2（把 else 拆到裡面）

- 這個是「外層 if 沒中，就進 else」，然後在 else 裡再判斷一次。

- 寫起來比較麻煩，但邏輯一樣。

In [ ]:
bills = []
hr_on = False
for word in myText2:
    if hr_on == False and word == "HR":
        hr_on = True
    else:
        if hr_on == True:
            bills.append(word)
            print(word)
            hr_on = False

6092
6093
6094


 寫法 3 用 regex 直接抓數字的版本

- 這個是想示範「一行 if」的寫法。

- 只要這個字裡面有數字，就丟進 bills。但這樣會抓到「所有有數字的字」，不一定是 HR 的號碼，所以下面才又寫了下一個。

In [ ]:
bills = []
hr_on = False
for word in myText2:
    bills.append(word) if re.search("[0-9]+", word) else None   ## 這個就可能找到其他的字。

print(bills)

['2', '6092', '6093', '38', '6094']


- 這裡改成「一定要 4 個數字」才收，所以比較接近 H.R. 6092 這種號碼。

- 但一樣，它不知道「前面是不是 HR」，所以會抓比較多。

In [ ]:
bills = []
hr_on = False
for word in myText2:
    bills.append(word) if re.search("\d\d\d\d", word) else None   ## 這個就可能找到其他的字。

print(bills)

['6092', '6093', '6094']


<>:4: SyntaxWarning: invalid escape sequence '\d'
<>:4: SyntaxWarning: invalid escape sequence '\d'
/tmp/ipython-input-995855285.py:4: SyntaxWarning: invalid escape sequence '\d'
  bills.append(word) if re.search("\d\d\d\d", word) else None   ## 這個就可能找到其他的字。


### 4. for–else 的示範

- 這段是要示範：Python 的 for 可以接一個 else:

- 這個 else 不是 if 的 else，是「for 正常跑完一次都沒有被 break」才會跑。所以要注意縮排位置。

In [ ]:
# Slide 3
bills = []
hr_on = False
for word in myText2:
    if hr_on == False and word == "HR":
        hr_on = True
    elif hr_on == True:
        bills.append(word)
        print(word)
        hr_on = False
else:  ## 但是要注意else的位置，如果把這個放在 for loop 裡，你會得到不同的東西。
    print("Thing to do after loop")

6092
6093
6094
Thing to do after loop


### 5. 用 index 來抓（range(...)）

- 這段跟前面一樣，只是把「for word in ...」改成「for index in ...」。

- 好處是：你知道現在是第幾個 token，有時候要看前一個、下一個就會比較方便。

In [ ]:
bills = []
hr_on = False
index = 0   ## easeir to do the text,

for index in range(0, len(myText2)):  ## you can try to look at token text, range(min, max):
    if hr_on == False and myText2[index] == "HR":
        hr_on = True
    elif hr_on == True:
        bills.append(myText2[index])
        hr_on = False
print(bills)

['6092', '6093', '6094']


- 這段更聰明：它直接看「前一個是不是 HR」，如果是，就把「現在這個」當作號碼。

- 這就是你旁邊寫的：用位置來抓、用 -1。

- 這個寫法不用開關 hr_on，很乾淨。

In [ ]:
bills = []
index = 0

for index in range(0, len(myText2)):
    if myText2[index-1] == "HR":   ##  用ＨＲ和 bill # 的位置來抓，用 index 也可以。 用 index 和 -1。 ###### 好用 ######
        bills.append(myText2[index])  ## 看看這邊沒有負一，就是法案號碼。

print(bills)

['6092', '6093', '6094']


### 6. continue / break 版本

In [ ]:
bills = []
hr_on = False
for word in myText2:
    if hr_on == False and word == "HR":
        hr_on = True
    elif hr_on == True:
        bills.append(word)
        print(word)
        hr_on = False
    continue
else:
    print("Thing to do after loop")

6092
6093
6094
Thing to do after loop


- 這裡是示範 break：一旦抓到第一個號碼就整個 loop 停掉，下面的 else 就不會跑。


In [ ]:
bills = []
hr_on = False
for word in myText2:
    if hr_on == False and word == "HR":
        hr_on = True
    elif hr_on == True:
        bills.append(word)
        print(word)
        hr_on = False
        break ## elif 就不會再跑了也不會有 else 一樣也是要注意 for loop的位置。
else:
    print("Thing to do after loop")

6092


### 7. add() 的錯誤 & try / except

- 「要怎麼抓錯」。

- bills 是 list，但你故意寫成 bills.add(word)。list 沒有 .add()，所以會噴錯。

- 但因為你用 try: ... except: ... 包住，所以程式不會整個停掉，會跑到 except 印出錯誤。

In [ ]:
bills = []
hr_on = False
for word in myText2:
    if hr_on == False and word == "HR":
        hr_on = True
    elif hr_on == True:
        bills.add(word)
        print(word)
        hr_on = False
else:
    print("Thing to do after loop")

AttributeError: 'list' object has no attribute 'add'

- catch exceptio, when error occurs. 有的時候我們需要知道錯誤發生，因為我們要去修改它。

In [ ]:
# Slide 5： catch exceptio, when error occurs. 有的時候我們需要知道錯誤發生，因為我們要去修改它。
bills = []
hr_on = False
for word in myText2:
    if hr_on == False and word == "HR":    #### 這邊有把hr_on 改成 False，different from previuos one.
        hr_on = True
    elif hr_on == True:
        try:
            bills.add(word)
            print(word)
            hr_on = False
        except:
            print("Error: " + word)
            hr_on = False
else:
    print("Bills: " + str(bills))

Error: 6092
Error: 6093
Error: 6094
Bills: []


- 把錯誤訊息印得更漂亮，並且使用 as err。

In [ ]:
bills = []
hr_on = False
for word in myText2:
    if hr_on == False and word == "HR":
        hr_on = True
    elif hr_on == True:
        try:
            bills.add(word)
            print(word)
            hr_on = False
        except (Exception) as err:
            print("Error for Bill " + word + ": " + str(err))。  #捕捉錯誤並將錯誤物件儲存在變數 err 裡，可印出或記錄詳細訊息
            hr_on = False
else:
    print("Bills: " + str(bills))

SyntaxError: invalid character '。' (U+3002) (ipython-input-1589351246.py, line 12)

- 這一版是「不回報錯誤」

In [ ]:
bills = []
hr_on = False
for word in myText2:
    if hr_on == False and word == "HR":
        hr_on = True
    elif hr_on == True:
        try:
            bills.add(word)
            print(word)
            hr_on = False
        except:
            pass

print("Bills: " + str(bills))

Bills: []


### 8. 自訂函式抓法案號碼

- 第一個 function

- 這個 function 很單純：如果第一個字是 HR，就回傳第二個字。

- 如果第一個字不是 HR，就什麼都不回傳（= 回傳 None）。

In [ ]:
## Slide 7
## Custom Functions

def get_bills(word1, word2):
    if word1 == "HR":
        bill = word2
        return bill  ##  需要 return 才會回報, 不然不會有任何產出

- 這行會拿到 "2414"。

In [ ]:
get_bills("HR", "2414")

'2414'

- 這行會拿到 None，因為第一個不是 HR。

In [ ]:
get_bills("Amendment", "2")  ## wrong argument, so you got nothing.

接著是把它放進迴圈：

- 這裡用 len(myText2)-1 是怕最後一個 token 沒有下一個，會 out of range。

- 每一圈都把 (現在的字, 下一個字) 丟進 function，function 若回傳東西，就收起來。

- 這個寫法很乾淨，也比較「模組化」，你上課應該就是要教這個

In [ ]:
bills = []
for i in range(0, len(myText2)-1):   ## out of range: to fix it -1. 不然最後一輪會沒東西，就會有 our of range.
    theBill = get_bills(myText2[i], myText2[i+1])   ## local list: theBill.
    if theBill != None:   ## if not not there. (so there is sth there. )
        bills.append(theBill)
else:
    print(bills)   ## out of range:

['6092', '6093', '6094']


### 9. 第二個 function：有參數名稱的

- 這個是示範「參數可以寫名字」跟「可以給預設值」。

- 所以你等一下才能使用不同的函數

In [ ]:
# Slide 8
def get_bills2(second="", first=""):   ## you can give it a name ,,, and then you have to feed your function differently.
    if first == "HR":
        bill = second
        return bill

- 有名字順序就不重要了。

In [ ]:
bills = []
for i in range(0, len(myText2)-1):
    theBill = get_bills2(first=myText2[i], \
                         second=myText2[i+1])
    if theBill != None:
        bills.append(theBill)
else:
    print(bills)

['6092', '6093', '6094']


- 這裡是示範「我不想 -1，也可以用 try/except 把最後一個處理掉」。

- 最後一個沒有 i+1 就會進 except。

In [ ]:
bills = []
for i in range(0, len(myText2)):
    try:   ## since you use try, so you do not need to -1
        theBill = get_bills2(first=myText2[i], \
                         second=myText2[i+1])
    except:
        print(i)
        print(myText2[i])
        theBill = get_bills2(first=myText2[i])    ### even if you skip the argument, you can still run it.

    if theBill != None:
        bills.append(theBill)
else:
    print(bills)

### 250 is the iternation, it is also the token that we tokenized. and the last thifs of it is .

250
.
['6092', '6093', '6094']


### 10. dateutil 解析日期

In [ ]:
### Perkins Ch. 9
# Slide 11
import dateutil
from dateutil import parser

- 匯入可以「猜日期格式」的套件。

- 一行一行示範：它真的可以辨識很多種寫法。

In [ ]:
parser.parse("Wednesday October 14, 2020")

datetime.datetime(2020, 10, 14, 0, 0)

In [ ]:
parser.parse("Oct 14 2020")   ## it takes a lot of different format!

datetime.datetime(2020, 10, 14, 0, 0)

In [ ]:
parser.parse("10/14/20")

datetime.datetime(2020, 10, 14, 0, 0)

In [ ]:
parser.parse("10/14/65")

datetime.datetime(2065, 10, 14, 0, 0)

In [ ]:
parser.parse("10/14/75")  ## it can tell if it is month first or day first

datetime.datetime(1975, 10, 14, 0, 0)

In [ ]:
parser.parse("14/10/15")

datetime.datetime(2015, 10, 14, 0, 0)

In [ ]:
parser.parse("9/10/15", dayfirst=True)

datetime.datetime(2015, 10, 9, 0, 0)

- 若在時間敘述時加入不同元素要用 fuzzy

In [ ]:
parser.parse("Date Argued Jan. 10, 2002")

ParserError: Unknown string format: Date Argued Jan. 10, 2002

In [ ]:
parser.parse("Date Argued Jan. 10, 2002", fuzzy=True)  ## if you want to include differet things.

datetime.datetime(2002, 1, 10, 0, 0)

這是把「一大串日期＋字」變成「純日期」的小工具：

fuzzy=True：就算字串裡有別的字也幫你找日期

轉成字串之後變成像 2020-10-14 00:00:00

用空格切，取第一段 → 就只剩 2020-10-14

所以你可以從「Date Argued Jan. 10, 2002」這種判決文字裡抓出真正的日期。

In [ ]:
def getDate(date):
    dt = parser.parse(date, fuzzy=True)
    dt = str(dt)
    dt = re.split(" ", dt)[0]  ## [0] the first thing I got. and if you do not the only date, if you take that off, you will get the whole thing fron "Date Argued... "
    return dt

In [ ]:
getDate("Date Argued Jan. 10, 2002")

### 11. lxml 解析 HTML

* 用「結構」來講

In [ ]:
# Slide 12
from lxml import html

- 內容是一整頁 Hinkle 的網頁。

- 重點是怎麼得到頁面裡的連結 link

In [ ]:
## Example HTML
## Triple quotes are the easiest
myHTML = """
 <!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Strict//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-strict.dtd">
<html xmlns="http://www.w3.org/1999/xhtml">
<head>
<title>Rachael K. Hinkle - Research</title>
<meta http-equiv="Content-Language" content="English" />
<meta http-equiv="Content-Type" content="text/html; charset=UTF-8" />
<link rel="stylesheet" type="text/css" href="style.css" media="screen" />
</head>
<body>

<div id="wrap">

<div id="header">
<h1><a href="#">Rachael K. Hinkle</a></h1>
    <h2>Department of Political Science </h2>
    <h2>  University at Buffalo, SUNY</h2>

</div>

<div class="left">


<h2><a href="#">Publications</a></h2>
<div class="articles">

<b>Peer Reviewed Journals</b> <br />
<br />

Rachael K. Hinkle, Michael J. Nelson, and Morgan L.W. Hazelton. 2019 "Deferring, Deliberating, or Dodging Review? Examining the Mechanisms Behind Panel Effects." <i> Journal of Law and Courts</i>, forthcoming. <br />
<ul>
<li>Data and replication materials are available <a href="research/JLC_Replication_Data.zip">here</a>. </li>
</ul>
<br />

Morgan L.W. Hazelton, Rachael K. Hinkle, and James F. Spriggs, II. 2019. "Leveraging the Content of Supreme Court Briefs to Evaluate Their Informational Value." 40(2) <i>Justice System Journal</i> 126-157.<br />
<ul>
<li>Data and replication materials are available <a href="research/HHS_JSJ_2019_replication.zip">here</a>. </li>
</ul>
<br />

Rachael K. Hinkle and Michael J. Nelson. 2018. "The Intergroup Foundations of Policy Influence." 71(4) <i>Political Research Quarterly</i> 729-742.<br />
<ul>
<li>Data and replication materials are available <a href="research/HN_PRQ2018_replicationFiles.zip">here</a>. </li>
</ul>

<br />

Rachael K. Hinkle and Michael J. Nelson. 2018. "How to Lose Cases and Influence People." 8 <i>Statistics, Politics, and Policy</i> 195-221.<br />
<ul>
<li>Profiled on <a href="http://www.scotusblog.com/2018/05/academic-highlight-hinkle-and-nelson-look-at-why-some-dissents-become-doctrine/">SCOTUSBlog</a></li>
<li> Profiled on <a href="https://qz.com/1289603/a-new-study-makes-the-case-for-being-a-sore-loser/">Quartz</a>.</li>
 <li>   Data and replication materials are available <a href="research/SPP_Replication.zip">here</a>. </li>
</ul>


<br />

Elizabeth A. Tillman and Rachael K. Hinkle. 2018. "Of Whites and Men: How Gender and Race Impact Authorship of Published and Unpublished Opinions in the U.S. Courts of Appeals." <i>Research and Politics</i>, forthcoming.<br />
<ul>
 <li>   Data and replication materials are available <a href="research/TH_R&R_replication_files.zip">here</a>.</li>
</ul>


<br />

Michael J. Nelson and Rachael K. Hinkle. 2018. "How Opinion Content Influences Legal Development." <i>Justice System Journal</i>, forthcoming.<br />
<ul>
<li>Data and replication materials are available <a href="research/CTLreplication.zip">here</a>. </li>
</ul>

<br />

Rachael K. Hinkle. 2017. <a href="research/Hinkle.PE.2017.JLC.docx">"Panel Effects and Opinion Crafting in the U.S. Courts of Appeals."</a> 5 <i>Journal of Law and Courts</i> 313-336.<br />
<ul>
<li>    Data and replication materials are available <a href="research/HinkleJLC2017ReplicationFiles.zip">here</a>. </li>
</ul>
<br />


Rachael K. Hinkle. 2016. <a href="research/Hinkle.LSR.2015.docx">"Strategic Anticipation of En Banc Review in the U.S. Courts of Appeals."</a> 50 <i>Law and Society Review</i> 383-414.<br />
<ul>
<li>    Data and replication materials are available <a href="research/HinkleLSR2016ReplicationFiles.zip">here</a>. </li>
</ul>
<br />


Rachael K. Hinkle and Michael J. Nelson. 2016. <a href="research/SPPQ.Transmission.Precedent.FINAL.pdf">"The Transmission of Legal Precedent Among State Supreme Courts in the Twenty-First Century."</a> 16 <i>State Politics and Policy Quarterly</i> 391-410.<br />
<ul>
<li>    Data and replication materials are available <a href="http://dx.doi.org/10.15139/S3/12125">here</a>. </li>
</ul>
<br />

Morgan L.W. Hazelton, Rachael K. Hinkle, and Jee Seon Jeon. 2016. <a href="research/HHJ.APR.pdf"> "Sound the Alarm? Judicial Decisions Regarding Publication and Dissent."</a> 44 <i>American Politics Review</i>  649-681.<br />
<ul>
 <li>   Data and replication materials are available <a href="research/HHJ_APR_Replication_Files.zip">here</a>. </li>
</ul>
<br />


Rachael K. Hinkle. 2015. <a href="http://dx.doi.org/10.1086/681059">"Legal Constraint in the U.S. Courts of Appeals."</a> <a href="research/Hinkle.JOP.2015.pdf">77 <i>Journal of Politics</i> 721-735.</a> <br />
<ul>
<li>Best Journal Article Award, APSA Law and Court Section 2016 </li>
<li>Data and replication materials are available <a href="http://dx.doi.org/10.7910/DVN/28929">here</a>. </li>
</ul>
<br />


Rachael K. Hinkle. 2015. <a href="http://onlinelibrary.wiley.com/doi/10.1111/ajps.12160/abstract">"Into the Words: Using Statutory Text to Explore the Impact of Federal Courts on State Policy Diffusion."</a> 59 <i>American Journal of Political Science</i> 1002-1021.<br />
<ul>
<li>    Data and replication materials are available <a href="http://dx.doi.org/10.7910/DVN/27063">here</a>. </li>
</ul>
<br />

Rachael K. Hinkle, Andrew D. Martin, Jonathan Shaub, and Emerson H. Tiller. 2012.
<a href="research/JLA.HMST.2012.pdf">"A Positive Theory and Empirical Analysis of Strategic Word Choice in District Court Opinions."</a>
4 <i>Journal of Legal Analysis</i> 407-444. <br />
 <br />

 Morgan L.W. Hazelton, Rachael K. Hinkle, and Andrew D. Martin. 2010. <a href="research/HHM2010.pdf">"On Replication and
     the Study of the Louisiana Supreme Court."</a> 10 <i>Global Jurist.</i> 85-91. <br />
 <ul>
<li>     Data and replication materials are available <a href="palmerReplication">here</a>. </li>
 </ul>
<br />

<b>Other Journals</b> <br />
<br />

Morgan L.W. Hazelton, Rachael K. Hinkle, and James F. Spriggs II. 2017. <a href="research/HHS.WashU.J.L.Policy.docx">"The Long and the Short of It: The Influence of Briefs on Outcomes in the Roberts Court."</a> 54 <i>Washington University Journal of Law Policy</i> 123-136. <br />

<br />

Peter J. Wiedenbeck, Rachael K. Hinkle, and Andrew D. Martin. 2013. <a
  href="research/InvisiblePension2.pdf">"Invisible Pension
  Investments."</a> 32 <i>Virginia Tax Review</i> 591-702.
<ul>
<li>  Web appendix is available <a href="http://invisiblepensioninvestments.wustl.edu">here</a>. </li>
</ul>
   <br />

Rachael K. Hinkle. 2004. <a href="research/ONU.article.pdf">"Sex, Race
  and Sour Grapes: A Look at the Sixth Circuit's Summary Judgment
  Jurisprudence in Title VII Hostile Work Environment Claims."</a> 30
  <i>Ohio Northern Univ. Law Review</i> 85. <br />
<br />

Rachael K. Hinkle. 2002. <a href="research/ONU.case.note.pdf">"INS
  v. St. Cyr."</a> 28
  <i>Ohio Northern Univ. Law Review</i> 815. <br />
<br />

<b>Blog Posts</b> <br />
<br />

Michael J. Nelson and Rachael K. Hinkle. March 13, 2018. <a href="http://wapo.st/2GqW2eQ">"Trump Appoints Lots of White Men to be Federal Judges. Here's Why it Matters."</a> The Monkey Cage.<br />
<br />

Rachael K. Hinkle, Morgan L.W. Hazelton, and Michael J. Nelson. May 26, 2017. <a href="http://bit.ly/2vYdxRz">"Getting to Know You - The Unifying Effects of Membership Stability."</a> SCOTUSBlog.<br />
<br />

Rachael K. Hinkle, Morgan L.W. Hazelton, and Jee Seon Jeon. July 6, 2016. <a href="http://blogs.lse.ac.uk/usappblog/2016/07/06/how-one-circuit-court-judge-can-stop-a-higher-court-from-establishing-a-legal-precedent/">"How One Circuit Court Judge Can Stop a Higher Court from Establishing a Legal Precedent."</a> London School of Economics United States Politics and Policy Blog.<br />
<br />


Rachael K. Hinkle. January 13, 2016. <a href="http://blogs.lse.ac.uk/usappblog/2016/01/13/how-states-learn-from-the-success-or-failure-of-other-states-laws-in-federal-courts/">"How States Learn from the Success or Failure of other States' Laws in Federal Courts."</a> London School of Economics United States Politics and Policy Blog.<br />
<br />


</div>


    <div class="right">

        <h2></h2>
        <ul>
            <p align="center">
            <br /><br />
            <img src="images/headshot2b.png" width="170" alt="Photo by Marlayna Demond" style="border: 0px solid #ccc;" />
            <br /><br />
            <small>
                Photo by <a href="http://marlaynaphotography.com/">Marlayna Demond</a>
            </small>
            </p>
        </ul>

        <h2>Categories: </h2>
        <ul>
            <li><a href="index.html">Home</a></li>
            <li><a href="cv.html">CV</a></li>
            <li><a href="research.html">Research</a></li>
            <li><a href="resources.html">Resources</a></li>
            <li><a href="courses.html">Courses</a></li>
            <li><a href="methods.html">Methods Training</a></li>
            <li><a href="legal.html">Legal Training</a></li>
            <li><a href="contact.html">Contact</a></li>
        </ul>

        <ul>
            <p align="center">
            <br /><br />
            <img src="images/cerlPowered.jpg" alt="CERL logo" style="border: 2px solid #bbb;" />
            <br /><br />
            </p>
        </ul>

    </div>

    <div style="clear: both;"> </div>

    <div id="footer">
        Designed by <a href="http://www.free-css-templates.com/">Free CSS Templates</a>, Thanks to <a href="http://www.openwebdesign.org/">custom web design</a></div>
</div>

</body>
</html>

"""

- 把文字版 HTML 轉成 lxml 能操作的樹狀結構。

- iterlinks() 會幫你把 HTML 裡所有的連結抓出來。

- 你先把它變成 ```list``` ，再看有幾個。

In [ ]:
myHTML2 = html.fromstring(myHTML)
myLinks = list(myHTML2.iterlinks())  ## get the links: iterlinks()
print(myLinks)
len(myLinks)

- 每一個 link 其實是一個 tuple，link[2] 就是 URL。

- 這裡在示範「怎麼一個個列出 href」。

In [ ]:
for link in myLinks:
    print(link[2])   ## and this is how you get the linke, from the 3rd position, which is [2]

### 12. 清洗 HTML

- 示範：可以用 clean_html 把多餘的東西清掉，或轉成比較乾淨的文字。

In [ ]:
# Slide 13
!pip install lxml_html_clean
from lxml_html_clean import Cleaner


# !pip install "lxml[html_clean]"
# from lxml.html.clean import Cleaner

In [ ]:
from lxml_html_clean import Cleaner

cleaner = Cleaner()
cleaner.javascript = True
cleaner.scripts = True
cleaner.style = True
cleaner.comments = True
cleaned_html = cleaner.clean_html(myHTML)
print(cleaned_html[:250])

要「完全去除 HTML 標籤，只留純文字」那就需要額外步驟。你可以用 lxml 或 BeautifulSoup 來解析後再取文字內容：

- 方式 1：搭配 lxml

In [ ]:
from lxml import html
from lxml_html_clean import Cleaner

cleaner = Cleaner()
cleaned_html = cleaner.clean_html(myHTML)

tree = html.fromstring(cleaned_html)
text = tree.text_content()  # 只留下純文字
print(text[:250])

### 13. BeautifulSoup 的版本

- 做的是跟 lxml 一樣的事，但是用 BS4 的寫法。

- findAll('a') 找所有超連結，再把 a['href'] 抽出來。

- 比較「lxml 寫法」跟「bs4 寫法」。

In [ ]:
# Slide 16
from bs4 import BeautifulSoup

In [ ]:
mySoup = BeautifulSoup(myHTML)


<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Strict//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-strict.dtd">
<html xmlns="http://www.w3.org/1999/xhtml">
<head>
<title>Rachael K. Hinkle - Research</title>
<meta content="English" http-equiv="Content-Language"/>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<link href="style.css" media="screen" rel="stylesheet" type="text/css"/>
</head>
<body>
<div id="wrap">
<div id="header">
<h1><a href="#">Rachael K. Hinkle</a></h1>
<h2>Department of Political Science </h2>
<h2>  University at Buffalo, SUNY</h2>
</div>
<div class="left">
<h2><a href="#">Publications</a></h2>
<div class="articles">
<b>Peer Reviewed Journals</b> <br/>
<br/>

Rachael K. Hinkle, Michael J. Nelson, and Morgan L.W. Hazelton. 2019 "Deferring, Deliberating, or Dodging Review? Examining the Mechanisms Behind Panel Effects." <i> Journal of Law and Courts</i>, forthcoming. <br/>
<ul>
<li>Data and replication materials are available <a href="research/JLC_Re

In [ ]:
[a['href'] for a in mySoup.findAll('a')]   ## a['href'] : for pull up links.  it is always this case in HTML.


#這裡的 a 是一個 <a> 標籤物件（BeautifulSoup 的 Tag 物件）。 a['href'] 表示取出這個標籤裡的 href 屬性，也就是「超連結網址」。

/tmp/ipython-input-106058537.py:1: DeprecationWarning: Call to deprecated method findAll. (Replaced by find_all) -- Deprecated since version 4.0.0.
  [a['href'] for a in mySoup.findAll('a')]   ## a['href'] : for pull up links.  it is always this case in HTML.


['#',
 '#',
 'research/JLC_Replication_Data.zip',
 'research/HHS_JSJ_2019_replication.zip',
 'research/HN_PRQ2018_replicationFiles.zip',
 'http://www.scotusblog.com/2018/05/academic-highlight-hinkle-and-nelson-look-at-why-some-dissents-become-doctrine/',
 'https://qz.com/1289603/a-new-study-makes-the-case-for-being-a-sore-loser/',
 'research/SPP_Replication.zip',
 'research/TH_R&R_replication_files.zip',
 'research/CTLreplication.zip',
 'research/Hinkle.PE.2017.JLC.docx',
 'research/HinkleJLC2017ReplicationFiles.zip',
 'research/Hinkle.LSR.2015.docx',
 'research/HinkleLSR2016ReplicationFiles.zip',
 'research/SPPQ.Transmission.Precedent.FINAL.pdf',
 'http://dx.doi.org/10.15139/S3/12125',
 'research/HHJ.APR.pdf',
 'research/HHJ_APR_Replication_Files.zip',
 'http://dx.doi.org/10.1086/681059',
 'research/Hinkle.JOP.2015.pdf',
 'http://dx.doi.org/10.7910/DVN/28929',
 'http://onlinelibrary.wiley.com/doi/10.1111/ajps.12160/abstract',
 'http://dx.doi.org/10.7910/DVN/27063',
 'research/JLA.HMST

- 完全去除 HTML 標籤，只留純文字」 那就需要額外步驟。

- 用  BeautifulSoup 來解析後再取文字內容：

In [ ]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(myHTML, "lxml")
text = soup.get_text(separator="\n", strip=True)
print(text[:250])

Rachael K. Hinkle - Research
Rachael K. Hinkle
Department of Political Science
University at Buffalo, SUNY
Publications
Peer Reviewed Journals
Rachael K. Hinkle, Michael J. Nelson, and Morgan L.W. Hazelton. 2019 "Deferring, Deliberating, or Dodging R


### 14. 編碼偵測、轉 ASCII

In [ ]:
!pip install chardet

In [ ]:
import chardet

# 假設你有一段 bytes（真正的編碼偵測要對 bytes 做，不能只對 str 做）
data = myText.encode("utf-8")  # 這裡先假裝它是 utf-8 的 bytes

result = chardet.detect(data)
print(result)

{'encoding': 'ascii', 'confidence': 1.0, 'language': ''}


In [ ]:
import chardet

# 英文
t1 = "Department of Political Science".encode("utf-8")
print(chardet.detect(t1))
# {'encoding': 'ascii', 'confidence': 1.0, 'language': ''}

# 中英文混合
t2 = "Department of Political Science 政治學系".encode("utf-8")
print(chardet.detect(t2))
# {'encoding': 'utf-8', 'confidence': 0.99, 'language': ''}


{'encoding': 'ascii', 'confidence': 1.0, 'language': ''}
{'encoding': 'utf-8', 'confidence': 0.938125, 'language': ''}


- 這裡是在示範「把有重音、特殊符號的字，通通轉成 ASCII」，有時候做 NLP 會這樣先清洗。

- encode("ascii", "ignore") 就是：轉不了的就丟掉。

In [ ]:
import unicodedata
myText3 = unicodedata.normalize("NFKD", myText).encode("ascii", "ignore")
print(myText3[:350])

In [ ]:
# 偵測編碼 → 轉換成 ASCII
import chardet
import unicodedata

# 兩段不同語系文字
text_1 = "Université de Montréal"  # 含法文字母 é
text_2 = "政治學系 Department of Political Science"  # 含中文

# 將文字編碼成 bytes（模擬不同來源資料）
data_1 = text_1.encode("utf-8")
data_2 = text_2.encode("utf-8")

# Step 1. 偵測編碼
print("Detected encoding for text_1:", chardet.detect(data_1))
print(Detected encoding for text_2:", chardet.detect(data_2))

# Step 2. 將 bytes 轉回文字
decoded_1 = data_1.decode("utf-8")
decoded_2 = data_2.decode("utf-8")

# Step 3. 正規化並轉為 ASCII（去除重音、中文）
ascii_1 = unicodedata.normalize("NFKD", decoded_1).encode("ascii", "ignore").decode("ascii")
ascii_2 = unicodedata.normalize("NFKD", decoded_2).encode("ascii", "ignore").decode("ascii")

# Step 4. 顯示結果
print("\nBefore → After (converted to ASCII):")
print(decoded_1, "→", ascii_1)
print(decoded_2, "→", ascii_2)


In [ ]:
text = "政治學系"
print(text.encode("big5"))
print(text.encode("big5").hex().upper())


In [ ]:
import chardet
import unicodedata

# Step 1. 模擬從 Big5 檔案讀取 bytes
raw_data = b'\xacF\xaav\xbe\xc7\xa8t' # "政治學系" in Big5

# Step 2. 偵測編碼
print(chardet.detect(raw_data))

# Step 3. 解碼成文字
text = raw_data.decode("big5")
print("Decoded text:", text)

# Step 4. 轉成 ASCII（移除中文）
ascii_text = unicodedata.normalize("NFKD", text).encode("ascii", "ignore").decode("ascii")
print("Converted to ASCII:", ascii_text)


### 15. API

https://dandelion.eu/

「實體連結 (entity linking)」。不是只看字面，而是會想：「這個字可能指的是哪些訊息」

抓取你給的訊息，連結到可能的訊息。

In [ ]:
import requests

API_TOKEN = "c88a87e6f3ac48229fa8f1090505eb99"

url = "https://api.dandelion.eu/datatxt/nex/v1/"
text = "Taiwan and the United States plan to deepen security cooperation."

params = {
    "text": text,
    "lang": "en",
    "token": API_TOKEN
}

resp = requests.get(url, params=params)  #把資料送給 API 並接收回應
data = resp.json() #解析伺服器的 JSON 回應
print(data)


{'time': 2, 'annotations': [{'start': 0, 'end': 6, 'spot': 'Taiwan', 'confidence': 0.8463, 'id': 25734, 'title': 'Taiwan', 'uri': 'http://en.wikipedia.org/wiki/Taiwan', 'label': 'Taiwan'}, {'start': 15, 'end': 28, 'spot': 'United States', 'confidence': 0.7119, 'id': 3434750, 'title': 'United States', 'uri': 'http://en.wikipedia.org/wiki/United_States', 'label': 'United States'}, {'start': 29, 'end': 33, 'spot': 'plan', 'confidence': 0.6382, 'id': 24918, 'title': "People's Liberation Army Navy", 'uri': 'http://en.wikipedia.org/wiki/People%27s_Liberation_Army_Navy', 'label': "People's Liberation Army Navy"}, {'start': 44, 'end': 64, 'spot': 'security cooperation', 'confidence': 0.6197, 'id': 1343423, 'title': 'Collective security', 'uri': 'http://en.wikipedia.org/wiki/Collective_security', 'label': 'Collective security'}], 'lang': 'en', 'timestamp': '2025-11-03T09:05:56.640'}


- 美化輸出

In [ ]:
import json
print(json.dumps(data, indent=2, ensure_ascii=False))

{
  "time": 2,
  "annotations": [
    {
      "start": 0,
      "end": 6,
      "spot": "Taiwan",
      "confidence": 0.8463,
      "id": 25734,
      "title": "Taiwan",
      "uri": "http://en.wikipedia.org/wiki/Taiwan",
      "label": "Taiwan"
    },
    {
      "start": 15,
      "end": 28,
      "spot": "United States",
      "confidence": 0.7119,
      "id": 3434750,
      "title": "United States",
      "uri": "http://en.wikipedia.org/wiki/United_States",
      "label": "United States"
    },
    {
      "start": 29,
      "end": 33,
      "spot": "plan",
      "confidence": 0.6382,
      "id": 24918,
      "title": "People's Liberation Army Navy",
      "uri": "http://en.wikipedia.org/wiki/People%27s_Liberation_Army_Navy",
      "label": "People's Liberation Army Navy"
    },
    {
      "start": 44,
      "end": 64,
      "spot": "security cooperation",
      "confidence": 0.6197,
      "id": 1343423,
      "title": "Collective security",
      "uri": "http://en.wikipedia.org/w

- 轉表格輸出

In [ ]:
import requests
import json
import pandas as pd

API_TOKEN = "c88a87e6f3ac48229fa8f1090505eb99"

url = "https://api.dandelion.eu/datatxt/nex/v1/"
text = "Taiwan and the United States plan to deepen security cooperation."

params = {
    "text": text,
    "lang": "en",
    "token": API_TOKEN
}

resp = requests.get(url, params=params)
data = resp.json()

# JSON 輸出
print(json.dumps(data, indent=2, ensure_ascii=False))

# 整理成表格
annotations = data.get("annotations", [])
results = []

for ann in annotations:
    results.append({
        "text": text,
        "spot": ann.get("spot"),
        "label": ann.get("label"),
        "confidence": round(ann.get("confidence", 0), 4)
    })

# 用 pandas 建 DataFrame
df = pd.DataFrame(results)

# 輸出的表格
print(df)

# 統計出現次數
print(df["label"].value_counts())


{
  "time": 2,
  "annotations": [
    {
      "start": 0,
      "end": 6,
      "spot": "Taiwan",
      "confidence": 0.8463,
      "id": 25734,
      "title": "Taiwan",
      "uri": "http://en.wikipedia.org/wiki/Taiwan",
      "label": "Taiwan"
    },
    {
      "start": 15,
      "end": 28,
      "spot": "United States",
      "confidence": 0.7119,
      "id": 3434750,
      "title": "United States",
      "uri": "http://en.wikipedia.org/wiki/United_States",
      "label": "United States"
    },
    {
      "start": 29,
      "end": 33,
      "spot": "plan",
      "confidence": 0.6382,
      "id": 24918,
      "title": "People's Liberation Army Navy",
      "uri": "http://en.wikipedia.org/wiki/People%27s_Liberation_Army_Navy",
      "label": "People's Liberation Army Navy"
    },
    {
      "start": 44,
      "end": 64,
      "spot": "security cooperation",
      "confidence": 0.6197,
      "id": 1343423,
      "title": "Collective security",
      "uri": "http://en.wikipedia.org/w